In [ ]:
import pandas as pd
import numpy as pd
import matplotlib.pyplot as plt


In [ ]:
train = pd.read_csv("house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("house-prices-advanced-regression-techniques/test.csv")